In [0]:
import keras
from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Activation, Flatten, Dropout, Reshape
from keras.layers import Convolution2D, UpSampling2D, BatchNormalization, Activation, ZeroPadding2D, MaxPooling2D
from keras.layers.convolutional import Conv2D, Conv2DTranspose, UpSampling2D, Convolution2D
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam, SGD
import numpy as np
import matplotlib.pyplot as plt
import random
from google.colab import drive
from tqdm import tqdm_notebook
from keras import backend as K

drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
class DCGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size, save_interval):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------
            
            vector_ind = np.random.permutation(X_train.shape[0]) # shuffle all training data
            nb_iter = int(X_train.shape[0]/batch_size) # nb_iter is the total number of iterations to complete one epoch given the batch size (=total number of samples / batch size)
            pbar = tqdm_notebook(range(nb_iter))
            for i in range(nb_iter):  # i is the iteration index
              idx = vector_ind[i*batch_size:(i+1)*batch_size] # grab a new set of 128 samples for this iteration # ind is the indices of the grabbed data

              # Select a random batch of images
              idx = np.random.randint(0, X_train.shape[0], batch_size) # generate 128 indices from [0 60000]
              imgs = X_train[idx] # get images according to selected indices

              # Sample noise and generate a batch of new images
              noise = np.random.normal(0, 1, (batch_size, self.latent_dim)) # sample noise of size 128*100
              gen_imgs = self.generator.predict(noise)

              # Train the discriminator (real classified as ones and generated as zeros)
              d_loss_real = self.discriminator.train_on_batch(imgs, valid)
              d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
              d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

              # ---------------------
              #  Train Generator
              # ---------------------

              # Train the generator (wants discriminator to mistake images as real)
              g_loss = self.combined.train_on_batch(noise, valid)

              # Plot the progress
              print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                  self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("/gdrive/My Drive/DCGAN/mnist_%d.png" % epoch)
        plt.close()

In [0]:
class LeNet():
  def __init__(self):
    # Input shape
    self.img_rows = 28
    self.img_cols = 28
    self.channels = 1
    self.img_shape = (self.img_rows, self.img_cols, self.channels)
    self.num_classes=10
  
  def get_lenet_model(self):
#     # model 1 https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py
#     lenet_model = Sequential()
#     lenet_model.add(Conv2D(32, kernel_size=(3, 3),activation='relu', input_shape=shape))
#     lenet_model.add(Conv2D(64, (3, 3), activation='relu'))
#     lenet_model.add(MaxPooling2D(pool_size=(2, 2)))
#     lenet_model.add(Dropout(0.25))
#     lenet_model.add(Flatten())
#     lenet_model.add(Dense(128, activation='relu'))
#     lenet_model.add(Dropout(0.5))
#     lenet_model.add(Dense(num_classes, activation='softmax'))
    # model 2 LeNet-5
    lenet_model = Sequential()
    lenet_model.add(Conv2D(6, kernel_size = 5,strides = 1,activation = 'relu',input_shape = shape))
    lenet_model.add(MaxPooling2D(pool_size =(2,2), strides = 2))
    lenet_model.add(Conv2D(16,kernel_size = 5,strides = 1,activation = 'relu'))
    lenet_model.add(MaxPooling2D(pool_size =(2,2), strides = 2))
    lenet_model.add(Flatten())
    lenet_model.add(Dense(120, activation = 'relu'))
    lenet_model.add(Dense(84, activation = 'relu'))
    lenet_model.add(Dense(num_classes, activation = 'softmax'))
    
    lenet_model.summary()
    return lenet_model
  
  def get_MNIST(self):
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255 # convert from [0 255] to [0 1]
    x_test /= 255 # convert from [0 255] to [0 1]
    # print('x_train shape:', x_train.shape)
    # print(x_train.shape[0], 'train samples')
    # print(x_test.shape[0], 'test samples')
    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)
  
  def train(self,epochs,batch_size):
    lenet_model=get_lenet_model(shape)
    lenet_model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
    lenet_model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
    score = lenet_model.evaluate(x_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

lenet=LeNet()
lenet.get_MNIST()
lenet.train(epochs=3,batch_size=128)

Train on 60000 samples, validate on 10000 samples
Epoch 1/3
  768/60000 [..............................] - ETA: 4:08 - loss: 2.0569 - acc: 0.2643

KeyboardInterrupt: ignored

In [0]:
dcgan = DCGAN()
dcgan.train(epochs=500, batch_size=128, save_interval=1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_45 (Conv2D)           (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu_17 (LeakyReLU)   (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_31 (Dropout)         (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_46 (Conv2D)           (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d_5 (ZeroPaddin (None, 8, 8, 64)          0         
_________________________________________________________________
batch_normalization_21 (Batc (None, 8, 8, 64)          256       
_________________________________________________________________
leaky_re_lu_18 (LeakyReLU)   (None, 8, 8, 64)          0         
__________

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 1.031109, acc.: 37.11%] [G loss: 1.069479]
0 [D loss: 0.404332, acc.: 82.03%] [G loss: 1.571806]
0 [D loss: 0.223618, acc.: 94.92%] [G loss: 1.747939]
0 [D loss: 0.180783, acc.: 97.27%] [G loss: 1.941305]
0 [D loss: 0.146595, acc.: 96.09%] [G loss: 2.091337]
0 [D loss: 0.257154, acc.: 90.62%] [G loss: 2.490319]
0 [D loss: 0.736050, acc.: 61.33%] [G loss: 3.577341]
0 [D loss: 0.787794, acc.: 57.03%] [G loss: 4.235783]
0 [D loss: 0.622469, acc.: 68.36%] [G loss: 3.787460]
0 [D loss: 0.493870, acc.: 75.78%] [G loss: 3.168262]
0 [D loss: 0.451169, acc.: 78.52%] [G loss: 2.343003]
0 [D loss: 0.330386, acc.: 88.67%] [G loss: 1.225200]
0 [D loss: 0.145703, acc.: 96.09%] [G loss: 0.603313]
0 [D loss: 0.089328, acc.: 98.44%] [G loss: 0.276218]
0 [D loss: 0.048159, acc.: 99.61%] [G loss: 0.210185]
0 [D loss: 0.060637, acc.: 99.22%] [G loss: 0.178563]
0 [D loss: 0.073754, acc.: 99.22%] [G loss: 0.181079]
0 [D loss: 0.141638, acc.: 93.36%] [G loss: 0.397482]
0 [D loss: 0.195121, acc.: 9

1 [D loss: 0.783236, acc.: 41.80%] [G loss: 0.893765]
1 [D loss: 0.773538, acc.: 41.41%] [G loss: 0.898798]
1 [D loss: 0.775539, acc.: 44.53%] [G loss: 0.947490]
1 [D loss: 0.748087, acc.: 45.31%] [G loss: 0.895880]
1 [D loss: 0.775674, acc.: 43.36%] [G loss: 0.849056]
1 [D loss: 0.753702, acc.: 45.31%] [G loss: 0.939597]
1 [D loss: 0.783819, acc.: 44.53%] [G loss: 0.891328]
1 [D loss: 0.798462, acc.: 41.80%] [G loss: 0.874069]
1 [D loss: 0.803534, acc.: 38.28%] [G loss: 0.863183]
1 [D loss: 0.765587, acc.: 42.97%] [G loss: 0.933992]
1 [D loss: 0.729123, acc.: 52.73%] [G loss: 0.909540]
1 [D loss: 0.775838, acc.: 42.58%] [G loss: 0.893396]
1 [D loss: 0.790444, acc.: 42.58%] [G loss: 0.884117]
1 [D loss: 0.759219, acc.: 46.48%] [G loss: 0.893168]
1 [D loss: 0.773555, acc.: 46.48%] [G loss: 1.023988]
1 [D loss: 0.771539, acc.: 47.66%] [G loss: 0.972954]
1 [D loss: 0.810899, acc.: 41.80%] [G loss: 0.905339]
1 [D loss: 0.773323, acc.: 43.36%] [G loss: 0.927326]
1 [D loss: 0.766503, acc.: 4

2 [D loss: 0.717201, acc.: 52.34%] [G loss: 0.897656]
2 [D loss: 0.755857, acc.: 42.19%] [G loss: 0.862391]
2 [D loss: 0.750454, acc.: 44.53%] [G loss: 0.839416]
2 [D loss: 0.758482, acc.: 44.92%] [G loss: 0.812067]
2 [D loss: 0.731236, acc.: 52.73%] [G loss: 0.858521]
2 [D loss: 0.748038, acc.: 45.31%] [G loss: 0.851227]
2 [D loss: 0.760550, acc.: 43.75%] [G loss: 0.859387]
2 [D loss: 0.761712, acc.: 39.84%] [G loss: 0.829439]
2 [D loss: 0.750466, acc.: 44.92%] [G loss: 0.873242]
2 [D loss: 0.742367, acc.: 43.75%] [G loss: 0.822662]
2 [D loss: 0.736324, acc.: 48.05%] [G loss: 0.859664]
2 [D loss: 0.736952, acc.: 47.27%] [G loss: 0.867239]
2 [D loss: 0.728822, acc.: 50.00%] [G loss: 0.890271]
2 [D loss: 0.758673, acc.: 46.88%] [G loss: 0.909194]
2 [D loss: 0.736684, acc.: 52.34%] [G loss: 0.847793]
2 [D loss: 0.741922, acc.: 41.80%] [G loss: 0.793015]
2 [D loss: 0.727754, acc.: 51.56%] [G loss: 0.845037]
2 [D loss: 0.731345, acc.: 50.00%] [G loss: 0.841191]
2 [D loss: 0.774056, acc.: 4

3 [D loss: 0.760774, acc.: 41.80%] [G loss: 0.819168]
3 [D loss: 0.714680, acc.: 51.17%] [G loss: 0.826387]
3 [D loss: 0.758898, acc.: 48.83%] [G loss: 0.857961]
3 [D loss: 0.725111, acc.: 48.05%] [G loss: 0.795271]
3 [D loss: 0.720124, acc.: 48.44%] [G loss: 0.844633]
3 [D loss: 0.751972, acc.: 43.75%] [G loss: 0.863552]
3 [D loss: 0.727242, acc.: 47.66%] [G loss: 0.860823]
3 [D loss: 0.745996, acc.: 45.31%] [G loss: 0.798015]
3 [D loss: 0.741504, acc.: 45.70%] [G loss: 0.831278]
3 [D loss: 0.717566, acc.: 46.48%] [G loss: 0.842546]
3 [D loss: 0.748930, acc.: 45.31%] [G loss: 0.805756]
3 [D loss: 0.755434, acc.: 46.48%] [G loss: 0.843381]
3 [D loss: 0.733377, acc.: 48.05%] [G loss: 0.875569]
3 [D loss: 0.774539, acc.: 40.62%] [G loss: 0.840645]
3 [D loss: 0.714472, acc.: 48.05%] [G loss: 0.837444]
3 [D loss: 0.731673, acc.: 47.66%] [G loss: 0.821941]
3 [D loss: 0.740015, acc.: 46.09%] [G loss: 0.857132]
3 [D loss: 0.706120, acc.: 54.30%] [G loss: 0.864385]
3 [D loss: 0.747929, acc.: 4

In [0]:
# inception score
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import os.path
import sys
import tarfile
from six.moves import urllib
import tensorflow as tf
import glob
import scipy.misc
import math
import sys

MODEL_DIR = '/tmp/imagenet'
DATA_URL = 'http://download.tensorflow.org/models/image/imagenet/inception-2015-12-05.tgz'
softmax = None

# Call this function with list of images. Each of elements should be a 
# numpy array with values ranging from 0 to 255.
def get_inception_score(images, splits=10):
  assert(type(images) == list)
  assert(type(images[0]) == np.ndarray)
  assert(len(images[0].shape) == 3)
  assert(np.max(images[0]) > 10)
  assert(np.min(images[0]) >= 0.0)
  inps = []
  for img in images:
    img = img.astype(np.float32)
    inps.append(np.expand_dims(img, 0))
  bs = 1
  with tf.Session() as sess:
    preds = []
    n_batches = int(math.ceil(float(len(inps)) / float(bs)))
    for i in range(n_batches):
        sys.stdout.write(".")
        sys.stdout.flush()
        inp = inps[(i * bs):min((i + 1) * bs, len(inps))]
        inp = np.concatenate(inp, 0)
        pred = sess.run(softmax, {'ExpandDims:0': inp})
        preds.append(pred)
    preds = np.concatenate(preds, 0)
    scores = []
    for i in range(splits):
      part = preds[(i * preds.shape[0] // splits):((i + 1) * preds.shape[0] // splits), :]
      kl = part * (np.log(part) - np.log(np.expand_dims(np.mean(part, 0), 0)))
      kl = np.mean(np.sum(kl, 1))
      scores.append(np.exp(kl))
    return np.mean(scores), np.std(scores)

# This function is called automatically.
def _init_inception():
  global softmax
  if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)
  filename = DATA_URL.split('/')[-1]
  filepath = os.path.join(MODEL_DIR, filename)
  if not os.path.exists(filepath):
    def _progress(count, block_size, total_size):
      sys.stdout.write('\r>> Downloading %s %.1f%%' % (
          filename, float(count * block_size) / float(total_size) * 100.0))
      sys.stdout.flush()
    filepath, _ = urllib.request.urlretrieve(DATA_URL, filepath, _progress)
    print()
    statinfo = os.stat(filepath)
    print('Succesfully downloaded', filename, statinfo.st_size, 'bytes.')
  tarfile.open(filepath, 'r:gz').extractall(MODEL_DIR)
  with tf.gfile.FastGFile(os.path.join(
      MODEL_DIR, 'classify_image_graph_def.pb'), 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    _ = tf.import_graph_def(graph_def, name='')
  # Works with an arbitrary minibatch size.
  with tf.Session() as sess:
    pool3 = sess.graph.get_tensor_by_name('pool_3:0')
    ops = pool3.graph.get_operations()
    for op_idx, op in enumerate(ops):
        for o in op.outputs:
            shape = o.get_shape()
            shape = [s.value for s in shape]
            new_shape = []
            for j, s in enumerate(shape):
                if s == 1 and j == 0:
                    new_shape.append(None)
                else:
                    new_shape.append(s)
            o.set_shape(tf.TensorShape(new_shape))
    w = sess.graph.get_operation_by_name("softmax/logits/MatMul").inputs[1]
    logits = tf.matmul(tf.squeeze(pool3, [1, 2]), w)
    softmax = tf.nn.softmax(logits)

_init_inception()

X_cifar10  = []
cifar10_imgs = glob.glob("./cifar10_inception_score_samples/*.png")
# load CIFAR10 sample images
for img in tqdm(cifar10_imgs):
    img = skimage.io.imread(img)[:, :, :3]
    X_cifar10.append(resize(img, (32, 32, 3))) 
plt.imshow(X_cifar10[0])
get_inception_score(X_cifar10[:-60], 10) # output mean and standard deviation